AI Chatbot Intent Classification is the process of identifying the purpose or goal behind a user's input in a conversation with a chatbot. It is a key component of Natural Language Understanding (NLU) in AI-driven chatbots and virtual assistants. *Intent Classification is important as it improves chatbot accuracy, enables efficient routing and enhances user experience.*

Working
*   User Input : User sends a message
*   Intent Detection : Chatbot analyzes the text to determine user's intent
*   Response Generation : Based on classified intent, chatbot provides an appropriate response

Techniques for intent classification
*   Rule-Based Matching ( Keyword pair, regex )
*   ML Models (NB,SVM,Random Forest)
*   Deep Learning and NLP Models (BERT,GPT,Transformer Based)
*   Hybrid Approaches (Combining rules + ML for better accuracy)



In [2]:
import pandas as pd
#Import Dataset from huggingface
splits = {'train': 'small/train-00000-of-00001.parquet',
          'validation': 'small/validation-00000-of-00001.parquet',
          'test': 'small/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/clinc/clinc_oos/" + splits["train"])

In [3]:
df.columns

Index(['text', 'intent'], dtype='object')

In [4]:
print(df["intent"].head())

0    108
1    108
2    108
3    108
4    108
Name: intent, dtype: int64


In [5]:
# If it's numeric, switch to string labels if available
if isinstance(df["intent"].iloc[0], int):
    print("Intent column is numeric! Look for a string column like 'intent_label'.")

In [6]:
#Extract training samples
train_texts=df['text'].tolist() #User Queries
train_labels=df['intent'].tolist() #Their corresponding intent names (as strings)

##Intent Classification Model

In [7]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pandas as pd

In [8]:
# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(train_labels)

In [ ]:
# Load SBERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

In [10]:
X_train = sbert_model.encode(train_texts, show_progress_bar=True)  # Convert text to embeddings

Batches:   0%|          | 0/238 [00:00<?, ?it/s]

In [ ]:
# Train classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

In [12]:
import pickle

with open("intent_label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

with open("intent_classifier.pkl", "wb") as f:
    pickle.dump(clf, f)

In [ ]:
# Load validation set
val_df = pd.read_parquet("hf://datasets/clinc/clinc_oos/small/validation-00000-of-00001.parquet")
val_texts = val_df["text"].tolist()
val_labels = val_df["intent"].tolist()
y_val = le.transform(val_labels)

# Encode validation texts
X_val = sbert_model.encode(val_texts, show_progress_bar=True)

# Predict and evaluate
y_pred = clf.predict(X_val)
# Get the target names (original string labels) from the fitted LabelEncoder
# Ensure target_names is a list of strings
target_names = [str(c) for c in le.classes_.tolist()]
print(classification_report(y_val, y_pred, target_names=target_names))

##NLU Pipeline (Intent recognition + Entity Extraction )

Goal is to build a NLU module that

*   Accepts input
*   Predicts intent using the model
*   Extracts entities from text (names,date,location etc.)




In [14]:
import pickle
from sentence_transformers import SentenceTransformer

# Load model and label encoder
with open("intent_classifier.pkl", "rb") as f:
    clf = pickle.load(f)

with open("intent_label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

# Load SBERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Intent prediction function
def predict_intent(user_input):
    # Step 1: Convert text to embedding
    embedding = sbert_model.encode([user_input])

    # Step 2: Predict intent class
    pred = clf.predict(embedding)[0]

    # Step 3: Convert class ID to intent name
    intent = le.inverse_transform([pred])[0]

    # Step 4: Get prediction confidence
    confidence = clf.predict_proba(embedding).max()

    return intent, confidence

In [16]:
intent, confidence = predict_intent("I want to book a flight to New York on Monday")
print(f"Intent: {intent} (Confidence: {confidence:.2f})")

Intent: 63 (Confidence: 0.58)


In [17]:
#Using NER
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

In [18]:
entities = extract_entities("I want to book a flight to New York on Monday")
print(entities)  # [('New York', 'GPE'), ('Monday', 'DATE')]

[('New York', 'GPE'), ('Monday', 'DATE')]
